# Heroes of Pymoli Data Analysis
*Observable Trend 1: The vast majority, 81%, of purchasing players identify as male and are between 20-24 years old (56.8%). The distribution of purchasers ages is approximately gaussian, centered around 20-24 years old.

*Observable Trend 2: Despite females being disproportionately lower represented in the purchasing population, those who purchase do so at similar rates (~1.3 purchases per player) as males. The average amount females who have made purchases spend is also within 5% of males. This suggests that of those HoP players who purchase items, there is approximately equal proclivities for puchase rates and amounts. Further statistical methods should be employed to validate.

*Observable Trend 3: Among the highest spending players, these players do not generally spend more per purchase than the average (~$4/purchase) and several spent less. Of the items purchased, the most commonly purchased are lower than the average price, but the highest grossing items are almost all (4/5) above the average purchase price. Further analysis is needed to determine the impact of price on purchasing decisions than is available herein.

In [66]:
#Declare and import libraries
import pandas as pd
import numpy as np
#Read data from local JSON into initial dataframe HoPdf
HoPdf = pd.read_json('purchase_data.json',encoding='UTF-8')

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


## Player Count

In [59]:
#Drop duplicate values from the SN column keeping the first instance to show unique purchasers
HoPdf_unique_sn = HoPdf.drop_duplicates(subset=['SN'],keep='first')
#Use the count method on the SN column (or any column) to return a count of purchasers
player_count = HoPdf_unique_sn['SN'].count()
#Use the DataFrame constructor to build this into a DF
player_count_df = pd.DataFrame(data={'Player Count':[player_count]},index=[0],columns=['Player Count'])
player_count_df

573 <class 'numpy.int32'>


,Player Count
0,573


## Purchasing Analysis

In [68]:
#Again drop duplicates from the Item ID field, retaining the first instance in a new DF
#Use count method on ItemID to count the number of unique item ID's purchased
HoPdf_unique_itemid = HoPdf.drop_duplicates(subset=['Item ID'],keep='first')
unique_weapon_int = HoPdf_unique_itemid['Item ID'].count()
#Using the mean method on the Price column of the original DF, store this value in a variable
mean_item_price = round(HoPdf['Price'].mean(),2)
#Use the sum() method on the ['Price'] column which will sum the cost of all transactions
#producing total revenue
total_revenue = round(HoPdf['Price'].sum(),2)
#Because the JSON is a record of in-game transactions over some period, the number of rows 
#in any category is the number of total transactions
total_purchases = HoPdf['Price'].count()
#Construct a DataFrame with these values
total_purch_data = pd.DataFrame(data={'Number of Unique Items':[unique_weapon_int],'Avg Purchase Price':[mean_item_price],
                                     'Total Number of Purchases':[total_purchases],'Total Revenue($)':[total_revenue]})
total_purch_data.head()

,Avg Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue($)
0,2.93,183,780,2286.33


## Gender Demographics

In [78]:
#Create a new data frame from the original dataframe with just screennames and genders
unique_sn_gender = HoPdf[['SN','Gender']]
#Drop duplicated SN values, but keep the first instance
unique_sn_gender = unique_sn_gender.drop_duplicates(subset=['SN'],keep='first')
unique_sn_gender_valcounts = unique_sn_gender['Gender'].value_counts() #series
#Use the total_players_int to convert counts to a rounded percentage and rename the column to show it is a percentage value
percent_gender = pd.DataFrame(round((unique_sn_gender['Gender'].value_counts() / total_players_int)*100))
percent_gender = percent_gender.rename(columns={'Gender':'Gender (%)'})
gender_demo = percent_gender.join(unique_sn_gender_valcounts,how='inner')
gender_demo = gender_demo.rename(columns={'Gender':'Total Count'})
gender_demo.head()

,Gender (%),Total Count
Male,81.0,465
Female,17.0,100
Other / Non-Disclosed,1.0,8


## Purchasing Analysis (Gender)

In [80]:
#Create a new data frame using groupby on Gender and count price
purchase_count_gender = HoPdf.groupby(['Gender'])['Price'].count()
purchase_count_gender = pd.DataFrame(purchase_count_gender)
#The below line is not needed in this
purchase_count_gender = purchase_count_gender.join(unique_sn_gender_valcounts,how='right')
purchase_count_gender = purchase_count_gender.join(round(HoPdf.groupby(['Gender'])['Price'].mean(),2),rsuffix='mean')
purchase_count_gender = purchase_count_gender.join(round(HoPdf.groupby(['Gender'])['Price'].sum(),2),rsuffix='sum')
purchase_count_gender['Normalized Totals'] = round(purchase_count_gender['Pricesum']/purchase_count_gender['Gender'],2)
purchase_count_gender = purchase_count_gender.rename(columns={'Price':'Count of Purchases','Pricemean':'Mean Price($)','Pricesum':'Total Revenue($)',
                                                              'Gender':'Player Count','Normalized Totals':'Normalized $/Player'})
purchase_count_gender

,Count of Purchases,Player Count,Mean Price($),Total Revenue($),Normalized $/Player
Male,633,465,2.95,1867.68,4.02
Female,136,100,2.82,382.91,3.83
Other / Non-Disclosed,11,8,3.25,35.74,4.47


## Age Demographics and Purchasing Analysis

In [91]:
#Create a list of age bins up to 51 since the max age (HoPdf['Age'].max() returns 45) is 45
age_bins = [0,9,14,19,24,29,34,39,44,49,54]
#Make bin labels that can be used to easily parse the data when tabulated into a df
bin_labels = ['0-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54']
#perform the cut on the original dataframe on the ['Age'] series
HoPdf['Age Group'] = pd.cut(HoPdf['Age'],bins=age_bins,labels=bin_labels)
#Use groupby to isolate data by the age bins
age_group_groupby = HoPdf.groupby(['Age Group'])
#As above, use the count, mean, and sum methods on relevant data series to establish
age_group_purchase_counts = age_group_groupby['SN'].count()
age_group_purchase_mean = round(age_group_groupby['Price'].mean(),2)
age_group_purchase_sum = round(age_group_groupby['Price'].sum(),2)
#Import SN's and AgeGroup to identify unique SN's and therefore players
unique_sn_age = HoPdf[['SN','Age Group']]
#Drop duplicated SN values, but keep the first instance
unique_sn_age = unique_sn_age.drop_duplicates(subset=['SN'],keep='first')
#Assign the counts of each age group
unique_sn_age = unique_sn_age['Age Group'].value_counts()
#Use the above to generate a percentage each age bin is of the overall purchasing population
age_group_pct = round((unique_sn_age/player_count)*100,2)
#generate a new dataframe to incorporate the above data
age_demographics_df = pd.DataFrame(age_group_purchase_counts)
age_demographics_df = age_demographics_df.rename(columns={'SN':'Purchase Count'})
age_demographics_df['% of Players'] = age_group_pct
age_demographics_df['Average Purchase Price ($)'] = age_group_purchase_mean
age_demographics_df['Total Revenue ($)'] = age_group_purchase_sum
age_demographics_df["Players"] = unique_sn_age
age_demographics_df['Normalized Totals ($/player)'] = round(age_demographics_df['Total Revenue ($)']/age_demographics_df['Players'],2)
age_demographics_df

,Purchase Count,% of Players,Average Purchase Price ($),Total Revenue ($),Players,Normalized Totals ($/player)
Age Group,,,,,,
0-9,28,3.32,2.98,83.46,19,4.39
10-14,35,4.01,2.77,96.95,23,4.22
15-19,133,17.45,2.91,386.42,100,3.86
20-24,336,45.20,2.91,978.77,259,3.78
25-29,125,15.18,2.96,370.33,87,4.26
30-34,64,8.20,3.08,197.25,47,4.20
35-39,42,4.71,2.84,119.40,27,4.42
40-44,16,1.75,3.19,51.03,10,5.10
45-49,1,0.17,2.72,2.72,1,2.72


## Top Spenders

In [11]:
#To identify the players (SN) with the highest cost, groupby SN then sum on Price
SN_groupby = HoPdf.groupby(by=['SN'])
#Join on price count, sum, and mean of the groupby object
HoPdf_with_total = HoPdf.join(SN_groupby['Price'].sum(),on='SN',how='inner',rsuffix='Total')
HoPdf_with_total = HoPdf_with_total.join(SN_groupby['Price'].count(),on='SN',how='inner',rsuffix='Count')
HoPdf_with_total = HoPdf_with_total.join(round(SN_groupby['Price'].mean(),2),on='SN',how='inner',rsuffix='Mean')
#Rename PriceTotal to Total Spent($) for clarity
#Sort on newly named column
HoPdf_with_total = HoPdf_with_total.rename(columns={'PriceTotal':'Total Spent($)','PriceCount':'Number of Purchases','PriceMean':'Average Purchase($)'})
HoPdf_with_total = HoPdf_with_total.sort_values(['Total Spent($)'],ascending=False)
#Drop duplicate SNs, keeping first instance to extract top 5 spenders
HoPdf_total_fixed = HoPdf_with_total.drop_duplicates(subset=['SN'],keep='first')
#Drop all rows outside the top 5
HoPdf_total_fixed_five = HoPdf_total_fixed[:5]
#Remove extraneous columns and set index to SN's
HoPdf_topfive = HoPdf_total_fixed_five.drop(labels=['Gender','Item ID','Item Name','Age Group','Age','Price'],axis=1)
HoPdf_topfive = HoPdf_topfive.set_index(['SN'])
HoPdf_topfive.head(5)

,Total Spent($),Number of Purchases,Average Purchase($)
SN,,,
Undirrala66,17.06,5,3.41
Saedue76,13.56,4,3.39
Mindimnya67,12.74,4,3.18
Haellysu29,12.73,3,4.24
Eoda93,11.58,3,3.86


## Most Popular Items

In [14]:
#To Identify the Most Popular Items, groupby 'Item ID'
ItemID_groupby = HoPdf.groupby(by=['Item ID'])
#Join on ID count, price sum
HoPdf_items = HoPdf.join(ItemID_groupby['Item ID'].count(),on='Item ID',how='inner',rsuffix='Count')
HoPdf_items = HoPdf_items.join(ItemID_groupby['Price'].sum(),on='Item ID',how='inner',rsuffix='Sum')
#Rename Columns
HoPdf_items_fixed = HoPdf_items.rename(columns={'Price':'Item Price($)','Item IDCount':'Times Purchased','PriceSum':'Total Purchase Value($)'})
#Drop duplicate Item ID's while keeping first instance
HoPdf_items_fixed = HoPdf_items_fixed.drop_duplicates(subset=['Item ID'],keep='first')
#Sort by Purchase Count to sort by Popularity
HoPdf_items_sorted = HoPdf_items_fixed.sort_values(['Times Purchased'],ascending=False)
HoPdf_pop_items = HoPdf_items_sorted[:5]
#Remove Unneeded Columns and set index to Item ID's
HoPdf_pop_items = HoPdf_pop_items.drop(labels=['Age', 'Gender','SN','Age Group'],axis=1)
HoPdf_pop_items = HoPdf_pop_items.set_index(['Item ID'])
HoPdf_pop_items.head()

,Item Name,Item Price($),Times Purchased,Total Purchase Value($)
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
84,Arcane Gem,2.23,11,24.53
175,Woeful Adamantite Claymore,1.24,9,11.16
13,Serenity,1.49,9,13.41
31,Trickster,2.07,9,18.63


## Most Profitable Items

In [15]:
#Use the above dataframes after dropping duplicates to sort on Total Purchase Value
HoPdf_items_sorted = HoPdf_items_fixed.sort_values(['Total Purchase Value($)'],ascending=False)
HoPdf_prof_items = HoPdf_items_sorted[:5]
#Remove Unneeded Columns and set index to Item ID's
HoPdf_prof_items = HoPdf_prof_items.drop(labels=['Age', 'Gender','SN','Age Group'],axis=1)
HoPdf_prof_items = HoPdf_prof_items.set_index(['Item ID'])
HoPdf_prof_items.head()

,Item Name,Item Price($),Times Purchased,Total Purchase Value($)
Item ID,,,,
34,Retribution Axe,4.14,9,37.26
115,Spectral Diamond Doomblade,4.25,7,29.75
32,Orenmir,4.95,6,29.70
103,Singed Scalpel,4.87,6,29.22
107,"Splitter, Foe Of Subtlety",3.61,8,28.88
